In [ ]:
# Import the function to get connect to the db
from snowexsql.db import get_db

# Import our class for the points table
from snowexsql.data import PointData

# Import a useful function to format that data into a dataframe
from snowexsql.conversions import query_to_geopandas

# Import some tools to build dates 
from datetime import date

# This is what you will use for all of hackweek to access the db
db_name = 'snow:hackweek@52.32.183.144/snowex'

# Using the function get_db, we receive 2 ways to interact with the database
engine, session = get_db(db_name)

In [ ]:
# Output the list of tables in the database 
engine.table_names()

In [ ]:
from snowexsql.data import PointData, ImageData

In [ ]:
# Pick a dataset
dataset = 'depth'

# Pick a date
collection_date = date(2020, 2, 7)

# The part inside the query function is what we want back, in this case all columns for the point data
qry = session.query(PointData)

# We then want to filter by the selected the data type depth.
qry = qry.filter(PointData.type == 'depth')

# Filter by a date
qry = qry.filter(PointData.date == collection_date)

#instrument 
#qry = qry.filter(PointData.instrument =='magnaprobe'| 'mesa')

# Limit it to a couple thousand 
qry = qry.limit(2000)

# Execute the query and convert to geopandas in one handy function
df = query_to_geopandas(qry, engine)

# how many did we retrieve?
print(f'{len(df.index)} records returned!')

In [ ]:
# Get the unique datanames in the table
results = session.query(PointData.type).distinct().all()
print('Available types = {}'.format(', '.join([r[0] for r in results])))

# Get the unique instrument in the table
results = session.query(PointData.instrument).distinct().all()
print('\nAvailable Instruments = {}'.format(', '.join([str(r[0]) for r in results])))

# Get the unique dates in the table
results = session.query(PointData.date).distinct().all()
print('\nAvailable Dates = {}'.format(', '.join([str(r[0]) for r in results])))

# Get the unique surveyors in the table
results = session.query(PointData.surveyors).distinct().all()
print('\nAvailable surveyors = {}'.format(', '.join([str(r[0]) for r in results])))


In [ ]:
# Its convenient to store a query like the following 
qry = session.query(PointData)

# Then filter on it to just depth profiles
qry = qry.filter(PointData.type == 'depth')

# protect ourselves from a lot of data
qry = qry.limit(5)

result = qry.all()
print(result)

Raster Data--DEM?

In [ ]:
# import the handy function to convert raster db results to rasterio
from snowexsql.conversions import raster_to_rasterio

# Import the SQL function to access PostGIS functions
from sqlalchemy.sql import func

# Import rasterio for Writing
import rasterio 

# Query 1 raster tile and convert it to a geotiff
result = session.query(func.ST_AsTiff(ImageData.raster == 'DEM'))

print(raster)

# Convert the dataset to a rasterio dataset
dataset = raster_to_rasterio(session, result)

# Copy the profile/tiff metadata (not to be confused with the database metadata)
profile = dataset[0].profile

# Write to a file 
#with rasterio.open('raster.tif', 'w', **profile) as dst:
    #dst.write(dataset[0].read(1), 1)

In [ ]:
DEM = ImageData.raster =='DEM'
print(DEM)

# Convert the dataset to a rasterio dataset
dataset = raster_to_rasterio(session, DEM)

# Copy the profile/tiff metadata (not to be confused with the database metadata)
#profile = dataset[0].profile

# Write to a file 
#with rasterio.open('raster.tif', 'w', **profile) as dst:
    #dst.write(dataset[0].read(1), 1)
